In [0]:
def get_gold_table_config(table_name):
    config_df = spark.sql(f"""
        SELECT source_system, gold_table_name, load_type, file_location, silver_schema_name, 
               dimension_type, fact_type, record_ts, record_is_active
        FROM gold_control_table
        WHERE gold_table_name = '{table_name}' AND record_is_active = true
    """)
    return config_df.collect()[0] if not config_df.isEmpty() else None

In [0]:
def extract_gold_data(file_location):
    return spark.read.format("delta").load(file_location)

In [0]:
def transform_gold_data(df, config):
    if config.load_type == 'DIM':
        if config.dimension_type == 'SCD1':
            return handle_scd_type_1(df, config.gold_table_name)  # Implement SCD Type 1 handling
        elif config.dimension_type == 'SCD2':
            return handle_scd_type_2(df, config.gold_table_name)  # Implement SCD Type 2 handling
    elif config.load_type == 'FCT':
        if config.fact_type == 'transaction':
            return handle_transaction_fact(df, config)
        elif config.fact_type == 'accumulating':
            return handle_accumulating_snapshot(df, config)
        elif config.fact_type == 'periodic':
            return handle_periodic_snapshot(df, config)
        elif config.fact_type == 'insert_only':
            return handle_insert_only_fact(df, config)
    return df

In [0]:
def load_to_gold_table(df, gold_table_name):
    df.write.format("delta").mode("overwrite").saveAsTable(gold_table_name)

In [0]:
# Get gold table names from the tracking table
def get_gold_table_names():
    gold_tables_df = spark.sql("""
        SELECT table_name 
        FROM tracking_table 
        WHERE layer = 'Silver'
    """)
    return [row.table_name for row in gold_tables_df.collect()]

In [0]:
def log_to_table(table_name, layer, log_message, error_flag=False):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        INSERT INTO logging_table (table_name, layer, log_message, log_timestamp, error_flag)
        VALUES ('{table_name}', '{layer}', '{log_message}', '{timestamp}', {error_flag})
    """)

In [0]:
from transform_gold_layer import handle_scd_type_2
def transform_gold_data(df, config):
    if config.load_type == 'DIM':
        # Handle SCD Type 2 for dimension tables
        handle_scd_type_2(df, config.gold_table_name)
        return spark.table(config.gold_table_name)  # Return updated dimension table
    elif config.load_type == 'FCT':
        # Transform for fact tables, e.g., aggregating data
        return df.groupBy("some_column").agg({"value_column": "sum"})
    return df

In [0]:
def update_gold_tracking_table(gold_table_name):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        MERGE INTO tracking_table AS t
        USING (SELECT '{gold_table_name}' AS table_name, 'Gold' AS layer, '{timestamp}' AS last_loaded) AS s
        ON t.table_name = s.table_name AND t.layer = s.layer
        WHEN MATCHED THEN
            UPDATE SET last_loaded = s.last_loaded
        WHEN NOT MATCHED THEN
            INSERT (table_name, layer, last_loaded) VALUES (s.table_name, s.layer, s.last_loaded)
    """)

In [0]:
def execute_gold_tasks(table_names):
    for table_name in table_names:
        config = get_gold_table_config(table_name)
        if config:
            source_system = config.source_system
            gold_table_name = config.gold_table_name
            load_type = config.load_type
            file_location = config.file_location
            
            try:
                # Extract data
                df = extract_gold_data(file_location)
                log_to_table(gold_table_name, "Gold", f"Successfully extracted data from {file_location}")

                # Transform data
                transformed_df = transform_gold_data(df, config)  # This handles SCD Type 2 as needed
                log_to_table(gold_table_name, "Gold", "Successfully transformed data")

                # Load data into gold table
                load_to_gold_table(transformed_df, gold_table_name)
                log_to_table(gold_table_name, "Gold", "Successfully loaded data into gold table")

                # Update tracking table
                update_gold_tracking_table(gold_table_name)
                log_to_table(gold_table_name, "Gold", "Successfully updated tracking table")

            except Exception as e:
                log_to_table(gold_table_name, "Gold", f"Error processing {table_name}: {e}", error_flag=True)
                print(f"Error processing {table_name}: {e}")

        else:
            log_to_table(table_name, "Gold", f"No configuration found for table: {table_name}", error_flag=True)
            print(f"No configuration found for table: {table_name}")

In [0]:
try:
    # Get the list of gold table names
    gold_table_names = get_gold_table_names()
    
    # Execute the gold tasks
    execute_gold_tasks(gold_table_names)

except Exception as e:
    log_to_table("Gold Execution", "Gold", f"Error during gold table operations: {str(e)}", error_flag=True)
    print(f"Error during gold table operations: {str(e)}")